In [79]:
import numpy as np
import pandas as pd
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure, ColumnDataSource
from bokeh.tile_providers import get_provider, Vendors
from bokeh.palettes import Reds
from bokeh.transform import linear_cmap,factor_cmap
from bokeh.layouts import row, column
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, NumeralTickFormatter

##### 1.2 - Import CSV file to pandas DataFrame

In [80]:
df = pd.read_csv('coord_data.csv')
df = df[df['County'].isin(["Delaware, NY","Sullivan, NY", "Ulster, NY", "Orange, NY", "Rockland, NY", "Broome, NY", "Chenango, NY", "Otsego, NY", ])]

display(df.head())

,County,New Positives,All Positives,New Tests,All Tests,Positive,co-ordinates,latitude,longitude,geometry
3,"Broome, NY",68,6306,2076,243869,3.275530,"(42.52092361450195, -74.30384826660156)",42.161977,-75.830283,POINT (-74.30384826660156 42.52092361450195)
8,"Chenango, NY",34,815,339,49176,10.029499,"(42.1686897277832, -75.87785339355469)",42.478024,-75.602241,POINT (-75.87785339355469 42.1686897277832)
12,"Delaware, NY",11,488,215,39954,5.116279,"(41.76803207397461, -75.05734252929688)",42.193987,-74.966728,POINT (-75.05734252929688 41.76803207397461)
35,"Orange, NY",223,19146,2725,327078,8.183486,"(42.305580139160156, -77.04928588867188)",41.402410,-74.306252,POINT (-77.04928588867188 42.30558013916016)
38,"Otsego, NY",19,840,540,54143,3.518519,"(42.70102310180664, -74.92680358886719)",42.629776,-75.028841,POINT (-74.92680358886719 42.70102310180664)


##### 1.3 - Create Coordinate Transform Function

For this particular visualisation, we will need to convert our latitudes and longitudes into Mercator coordinates. To do this we will use a function ('borrowed' from my friend [Nadine Amersi-Belton](https://github.com/nadinezab/kc-house-prices-prediction)) to perform this transformation.

In [81]:
# Define function to switch from lat/long to mercator coordinates
def x_coord(x, y):
    
    lat = x
    lon = y
    
    r_major = 6378137.000
    x = r_major * np.radians(lon)
    scale = x/lon
    y = 180.0/np.pi * np.log(np.tan(np.pi/4.0 + 
        lat * (np.pi/180.0)/2.0)) * scale
    return (x, y)

# Define coord as tuple (lat,long)
df['coordinates'] = list(zip(df['latitude'], df['longitude']))


# Obtain list of mercator coordinates
mercators = [x_coord(x, y) for x, y in df['coordinates'] ]


##### 1.4 - Add Mercator Coordinates to DataFrame

Now we will add those coordinates to our DataFrame.

In [82]:
# Create mercator column in our df
df['mercator'] = mercators

# Split that column out into two separate cols - mercator_x and mercator_y
df[['mercator_x', 'mercator_y']] = df['mercator'].apply(pd.Series)

# Drop 'geometry' column 
df = df.drop(columns=['geometry'])

# Examine our modified DataFrame
df.head()


,County,New Positives,All Positives,New Tests,All Tests,Positive,co-ordinates,latitude,longitude,coordinates,mercator,mercator_x,mercator_y
3,"Broome, NY",68,6306,2076,243869,3.275530,"(42.52092361450195, -74.30384826660156)",42.161977,-75.830283,"(42.1619773, -75.8302833)","(-8441388.523665678, 5185273.823934164)",-8.441389e+06,5.185274e+06
8,"Chenango, NY",34,815,339,49176,10.029499,"(42.1686897277832, -75.87785339355469)",42.478024,-75.602241,"(42.478023799999995, -75.6022407)","(-8416002.937554501, 5232856.240408601)",-8.416003e+06,5.232856e+06
12,"Delaware, NY",11,488,215,39954,5.116279,"(41.76803207397461, -75.05734252929688)",42.193987,-74.966728,"(42.1939865, -74.9667281)","(-8345257.998529794, 5190082.118944368)",-8.345258e+06,5.190082e+06
35,"Orange, NY",223,19146,2725,327078,8.183486,"(42.305580139160156, -77.04928588867188)",41.402410,-74.306252,"(41.4024096, -74.3062522)","(-8271734.157660564, 5071879.412357982)",-8.271734e+06,5.071879e+06
38,"Otsego, NY",19,840,540,54143,3.518519,"(42.70102310180664, -74.92680358886719)",42.629776,-75.028841,"(42.6297762, -75.028841)","(-8352172.374929487, 5255788.717128242)",-8.352172e+06,5.255789e+06


### 2. Create Visualisation with Bokeh

##### 2.1 - Create Visualisation

The steps are described with comments below. Full explanations of each step can be found in the [associated article](https://towardsdatascience.com/creating-an-interactive-map-in-python-using-bokeh-and-pandas-f84414536a06).

This creates a [.html file](./output/avocado.html) which we can embed in dashboards or on [web pages](https://www.craigdoesdata.de/blog/avocadobokeh.html) as we please. The use of the 'show' function at the end also allows the visualisation to be displayed within our notebook.

In [97]:
# Create map with Bokeh

# Select tile set to use
chosentile = get_provider(Vendors.CARTODBPOSITRON_RETINA)

myReds = ['#FFCCBB', '#FF6C5C', '#F85C4D', '#E84C3D', '#D83C2D', '#BF0000']

# Choose palette
palette = myReds

# Reds[3].reverse()

# Tell Bokeh to use df as the source of the data
source = ColumnDataSource(data=df)

# Define color mapper - which column will define the colour of the data points
color_mapper = linear_cmap(field_name = 'Positive', palette = palette, low = df['Positive'].min(), high = df['Positive'].max())

# Set tooltips - these appear when we hover over a data point in our map, very nifty and very useful
nan_color = '#d9d9d9'
tooltips = [("County","@County"),("% Positive","@Positive")]


# Create figure
p = figure(title = 'COVID % Posivity Rates - Sullivan County Area', x_axis_type="mercator", y_axis_type="mercator", 
           x_axis_label = 'Longitude', y_axis_label = 'Latitude', tooltips = tooltips, plot_width=1600, plot_height=800,
           x_range=(-500000, 3500000), y_range=(-1000000, 7000000))
           

# Add map tile
p.add_tile(chosentile)


# Add points using mercator coordinates
p.circle(x = 'mercator_x', y = 'mercator_y', color = color_mapper, source=source, size=40, fill_alpha = 0.8)

#Defines color bar
color_bar = ColorBar(color_mapper=color_mapper['transform'], 
                     formatter = NumeralTickFormatter(format='0.0[0000]'), 
                     label_standoff = 13, width=8, location=(0,0))

# Set color_bar location
p.add_layout(color_bar, 'right')

# Display in notebook
output_notebook()

# Save as HTML
output_file('SulCOVID.html', title='COVID % Posivity Rates - Sullivan County Area')

# Show map
show(p)

Loading BokehJS ...